In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?
pd.set_option('display.max_columns', None) 

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca
import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time

from calitp import get_engine

engine = get_engine()
connection = engine.connect()

Creating trips per am peak / midday / pm / other by stop

In [ ]:
# set testing parameters - 1 operator, 1 weekday 
analysis_dt = dt.date(2022,6,1)

analysis_operator = 300

In [ ]:
help(shared_utils.gtfs_utils.get_stop_times)

In [ ]:
# pull trips per stop for each time bucket
## right now returning df, but consider returning dask df 


stoptimes_ampeak = (shared_utils.gtfs_utils.get_stop_times(
    selected_date = analysis_dt,
    itp_id_list = [analysis_operator],
    departure_hours = (6, 10),
    get_df = True
)
    >> group_by(_.calitp_itp_id,_.stop_id)
    >> summarize(n_trips_ampeak = _.trip_id.count())
                   )

stoptimes_midday = (shared_utils.gtfs_utils.get_stop_times(
    selected_date = analysis_dt,
    itp_id_list = [analysis_operator],
    departure_hours = (10, 15),
    get_df = True
)
    >> group_by(_.calitp_itp_id,_.stop_id)
    >> summarize(n_trips_midday = _.trip_id.count())
                   )

stoptimes_pmpeak = (shared_utils.gtfs_utils.get_stop_times(
    selected_date = analysis_dt,
    itp_id_list = [analysis_operator],
    departure_hours = (15, 19),
    get_df = True
)
    >> group_by(_.calitp_itp_id,_.stop_id)
    >> summarize(n_trips_pmpeak = _.trip_id.count())
                   )

# join by stop id
stoptimes_all = (stoptimes_ampeak
                 >> full_join(_,stoptimes_midday, on = ["calitp_itp_id","stop_id"])
                 >> full_join(_,stoptimes_pmpeak, on = ["calitp_itp_id","stop_id"])
                )

del stoptimes_ampeak
del stoptimes_midday
del stoptimes_pmpeak

In [ ]:
type(stoptimes_all)